In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import json
from pandas import json_normalize
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium import Choropleth
import matplotlib
from datetime import datetime 
# GetCompasData로 주피터 노트북 경로에 파일 저장
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
matplotlib.rcParams['font.family']="Malgun Gothic"
import requests
from bs4 import BeautifulSoup
import glob, os
import urllib
from urllib.parse import urlencode, quote_plus
import requests
from bs4 import BeautifulSoup
from pandas import json_normalize
import xml.dom.minidom

# 약국 주소 가져오기

In [33]:
def 약국매핑(지역):
    ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
    open_url = 'http://apis.data.go.kr/B552657/ErmctInsttInfoInqireService/getParmacyListInfoInqire'

    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                    quote_plus('Q0') : 지역,
                                   quote_plus('pageNo') : '1',
                                   quote_plus('numOfRows') : '10000' })
    open_url_get = open_url + queryParams

    res = requests.get(open_url_get)
    datas = BeautifulSoup(res.content,"html.parser").find_all("item")
    기관주소 = [] ; 기관명 = [] ; 경도 = [] ; 위도 = []

    for data in datas:
        기관주소.append(data.find("dutyaddr").get_text())
        기관명.append(data.find("dutyname").get_text())
        try:
            경도.append(data.find("wgs84lon").get_text()) 
        except: 경도.append(0)
        try:
            위도.append(data.find("wgs84lat").get_text()) 
        except: 위도.append(0)

    df = pd.DataFrame(columns = ["기관주소","기관명","경도","위도"])

    df["기관주소"] = 기관주소
    df["기관명"] = 기관명
    df["경도"] = 경도
    df["위도"] = 위도
    return df
    

In [44]:
약국_수원 = 약국매핑('수원')
약국_서울 = 약국매핑('서울특별시')
약국_진주 = 약국매핑('진주시')
약국_창원 = 약국매핑('창원시')

In [55]:
약국_3개시 = pd.concat([약국_서울,약국_진주,약국_창원])
약국_3개시

,기관주소,기관명,경도,위도
0,"서울특별시 영등포구 양평로 24, (당산동6가)",100세약국,126.903727777777775,37.533266666666670
1,"서울특별시 중구 서소문로 116, 유원빌딩 1304호 (서소문동)",13층약국,126.974007434429,37.5627316675026
2,"서울특별시 성동구 왕십리로 410, K동 106호 (하왕십리동, 센트라스)",153온누리약국,127.02624836755226,37.56532756147758
3,서울특별시 관악구 남부순환로 1832 1층,1번약국,126.9534959421,37.480704569
4,서울특별시 구로구 구로동로 132 (구로동),1번약국,126.88420201358698,37.49042028999001
...,...,...,...,...
425,경상남도 창원시 마산회원구 3.15대로 483,회원약국,128.5749229347276,35.217522566678134
426,"경상남도 창원시 성산구 대정로 138 (대방동, 대방대동아파트)",훼미리약국,128.70762135383123,35.21174512484588
427,"경상남도 창원시 성산구 가양로124번길 5-3, 101호 (대방동,경남빌딩)",희망약국,128.70713007182505,35.2090331750318
428,"경상남도 창원시 진해구 천자로 411, 105호(풍호동,경동프라자)",희망약국,128.7098285995319,35.1353184114902


In [82]:
len(약국_3개시[약국_3개시['위도']==0])

52

In [85]:
len(약국_수원[약국_수원['위도']==0])

4

# 병원 주소 가져오기

In [75]:
def 병원매핑(지역):
    ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
    open_url = 'http://apis.data.go.kr/B552657/HsptlAsembySearchService/getHsptlMdcncListInfoInqire'

    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                    quote_plus('Q0') : 지역,
                                   quote_plus('pageNo') : '1',
                                   quote_plus('numOfRows') : '10000' })
    open_url_get = open_url + queryParams

    res = requests.get(open_url_get)
    datas = BeautifulSoup(res.content,"html.parser").find_all("item")
    기관주소 = [] ; 기관명 = [] ; 경도 = [] ; 위도 = []

    for data in datas:
        기관주소.append(data.find("dutyaddr").get_text())
        기관명.append(data.find("dutyname").get_text())
        try:
            경도.append(data.find("wgs84lon").get_text()) 
        except: 경도.append(0)
        try:
            위도.append(data.find("wgs84lat").get_text()) 
        except: 위도.append(0)

    df = pd.DataFrame(columns = ["기관주소","기관명","경도","위도"])

    df["기관주소"] = 기관주소
    df["기관명"] = 기관명
    df["경도"] = 경도
    df["위도"] = 위도
    return df
    

In [76]:
병원_수원 = 병원매핑('수원')
병원_서울 = 병원매핑('서울특별시')
병원_진주 = 병원매핑('진주시')
병원_창원 = 병원매핑('창원시')

In [81]:
병원_3개시 = pd.concat([병원_서울,병원_진주,병원_창원])
병원_3개시

,기관주소,기관명,경도,위도
0,"서울특별시 동작구 상도로 170, 2층 (상도동)",(사)한국한센복지협회서울특별시지부부설의원,126.93803709245813,37.50407680682926
1,서울특별시 강남구 봉은사로 612 (삼성동),(의)가산의료재단광동한방병원,127.06214779521632,37.514279685612216
2,"서울특별시 강남구 테헤란로 513 (삼성동, B1일부,4,9,10층,테헤란로81길1...",(의)미래의료재단리드림의원,127.05888980410002,37.508024781100005
3,"서울특별시 강남구 테헤란로 513 (삼성동, 지하1층일부)",(의)미래의료재단미래치과의원,127.05888980410002,37.508024781100005
4,"서울특별시 강남구 도산대로 442 (청담동, 2층일부)",(의)성광의료재단 차움한의원,127.04415941273409,37.52334106045811
...,...,...,...,...
1316,"경상남도 창원시 성산구 원이대로393번길 25, 케이프타운 2-4층 일부, 지하1-...",희연요양병원,128.66253810302106,35.23996915376772
1317,"경상남도 창원시 성산구 원이대로393번길 25, 5층 5001호 (반지동, 케이프타운)",희연의원,128.6625599049,35.2399616614
1318,"경상남도 창원시 진해구 충장로 121, 203호 (여좌동)",흰돌치과의원,128.66537471633498,35.152540139557345
1319,"경상남도 창원시 진해구 진해대로 999, 201호 (자은동, 하이존)",흰토끼의원,128.70781238277007,35.14026576086863


In [83]:
len(병원_3개시[병원_3개시['위도']==0])

32

In [84]:
len(병원_수원[병원_수원['위도']==0])

1